In [3]:
import yaml

with open("../cfg.yaml", 'r') as stream:
    cfg = yaml.safe_load(stream)
    
if not cfg['api_key'] or not cfg['workspace'] or not cfg['project'] or not cfg['version']:
  print("Missing configuration data")
  exit(1)

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/Users/serhiipostupaiev/projects/yolov8-research/yolov5


In [3]:
# clone project repository and install dependencies

!git clone https://github.com/ultralytics/yolov5.git # clone
%cd {HOME}/yolov5
%pip install -r requirements.txt # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.11.7 torch-2.2.1 CPU


Setup complete ✅ (10 CPUs, 32.0 GB RAM, 272.8/460.4 GB disk)


In [5]:
# download pretrained weights

from utils.downloads import attempt_download

p5 = ['n']  # P5 models
cls = [f'{x}-seg' for x in p5]  # segmentation models

for x in cls:
    attempt_download(f'{HOME}/yolov5/weights/yolov5{x}.pt')

100%|██████████| 4.11M/4.11M [00:00<00:00, 5.96MB/s]



In [10]:
%cd {HOME}/yolov5 
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=cfg['api_key'])
project = rf.workspace(cfg['workspace']).project(cfg['project'])
dataset = project.version(cfg['version']).download('yolov5')

/Users/serhiipostupaiev/projects/yolov8-research/yolov5/yolov5
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cameraman-segmentation-8 in yolov7pytorch:: 100%|██████████| 5574/5574 [00:00<00:00, 10534.63it/s]


In [11]:
%cat {dataset.location}/data.yaml

names:
- cameraman
nc: 1
roboflow:
  license: CC BY 4.0
  project: cameraman-segmentation
  url: https://universe.roboflow.com/cv-wc4ti/cameraman-segmentation/dataset/8
  version: 8
  workspace: cv-wc4ti
test: ../test/images
train: cameraman-segmentation-8/train/images
val: cameraman-segmentation-8/valid/images


In [4]:
%cd {HOME}/yolov5 
!python segment/train.py \
  --imgsz 854 \
  --batch-size 16 \
  --epochs 5 \
  --data {dataset.location}/data.yaml \
  --weights yolov5s-seg.pt \
  --name custom-dataset \
  --hyp ./hyp.yaml

/Users/serhiipostupaiev/projects/yolov8-research/yolov5/yolov5


/Users/serhiipostupaiev/projects/yolov8-research/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


segment/train: weights=yolov5s-seg.pt, cfg=, data={dataset.location}/data.yaml, hyp=./hyp.yaml, epochs=5, batch_size=16, imgsz=854, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train-seg, name=custom-dataset, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, mask_ratio=4, no_overlap=False
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
Traceback (most recent call last):
  File "/Users/serhiipostupaiev/projects/yolov8-research/yolov5/yolov5/segment/train.py", line 760, in <module>
    main(opt)
  File "/Users/serhiipostupaiev/projects/yolov8-research/yolov5/yolov5/segment/train.py", line 611, in main
    check_file(opt.data),
    ^^^^^^^^^^^^^^^^^^^^
  File "/User